# Reference structure definitions

Fitting potentials using force-matching methodology requires defining reference atomic structures with corresponding target energies, per-atom forces and system-wide pressures typically obtained through DFT. This Notebook outlines how to define such values for use with the iprPy_fit code.

In [1]:
from pathlib import Path

import numpy as np

import yabadaba
import iprPy_fit

import atomman as am

uc = yabadaba.unitconvert

/home/lmh1/conda/envs/ipr/lib/python3.13/site-packages/numpy/_core/getlimits.py:552: UserWarning: Signature b'\x00\xd0\xcc\xcc\xcc\xcc\xcc\xcc\xfb\xbf\x00\x00\x00\x00\x00\x00' for <class 'numpy.longdouble'> does not match any known type: falling back to type probe function.
This warnings indicates broken support for the dtype!
  machar = _get_machar(dtype)


## 1. ReferenceStructure records

For convenience, a ReferenceStructure class is included that generates and reads reference_structure data records. These records collect all of the reference data for a single atomic structure and can be saved in either JSON or XML format. Such records can be loaded individually from local files, or placed in a compatible database and queried.

### 1.1. Build new records

New records can be defined by specifying values for the following class attributes.

- __name__ is a name for the record, typically what you want to save it as (without the .json or .xml extensions)
- __system__ is the atomic system represented as an atomman.System. This collects box information, per-atom atom types, positions and forces.
- __E_pot_total__ is the system's reference total potential energy.
- __E_pot_atom__ is the system's reference per-atom potential energy.
- __P_xx__ is the system's reference xx pressure.
- __P_yy__ is the system's reference yy pressure.
- __P_zz__ is the system's reference zz pressure.

#### 1.1.1. Create new empty record

In [2]:
record = yabadaba.load_record('reference_structure')

# Alternatively, init directly from the class in iprPy_fit
# record = iprPy_fit.record.ReferenceStructure.ReferenceStructure()

#### 1.1.2. Assign basic values.

yabadaba.unitconvert.set_in_units() converts the values from the units given into compatible working units.

Name is required, but the energy and pressure values are all optional.  If any energy or pressure values are not given, then the fitting process will simply not use them.

In [3]:
record.name = 'test-define'
record.E_pot_total = uc.set_in_units(-37.039999835953154, 'eV')
record.E_pot_atom = uc.set_in_units(-4.629999979494144, 'eV')
record.P_xx = uc.set_in_units(-0.17257378435893145, 'MPa')
record.P_yy = uc.set_in_units(-0.0645693145365454, 'MPa')
record.P_zz = uc.set_in_units(-0.06456931454719021, 'MPa')

#### 1.1.3. Add system information in the form of an atomman.System.

The atomman.load() method can be used to read in atomic system information from a variety of file formats (LAMMPS data, LAMMPS dump, poscar, CIF, JSON using atomman schemas, ...) or from alternate Python object representations (ase.Atoms, pymatgen.Structure, ...). See the atomman documentation for more information https://www.ctcms.nist.gov/potentials/atomman/tutorial/1.4._Load_and_dump_conversions.html.

In [4]:
# Example poscar content
poscar = """
1.0000000000000e+00
5.4335655707252e+00 0.0000000000000e+00 0.0000000000000e+00
0.0000000000000e+00 5.4335601371651e+00 0.0000000000000e+00
0.0000000000000e+00 0.0000000000000e+00 5.4335601371651e+00
Si
8 
direct
0.0000000000000e+00 0.0000000000000e+00 0.0000000000000e+00
5.0000000000000e-01 5.0000000000000e-01 0.0000000000000e+00
5.0000000000000e-01 0.0000000000000e+00 5.0000000000000e-01
0.0000000000000e+00 5.0000000000000e-01 5.0000000000000e-01
2.5000000000000e-01 2.5000000000000e-01 2.5000000000000e-01
2.5000000000000e-01 7.5000000000000e-01 7.5000000000000e-01
7.5000000000000e-01 2.5000000000000e-01 7.5000000000000e-01
7.5000000000000e-01 7.5000000000000e-01 2.5000000000000e-01
"""

# Load in the content (can alternately pass in path to POSCAR file)
record.system = am.load('poscar', poscar)
print(record.system)

avect =  [ 5.434,  0.000,  0.000]
bvect =  [ 0.000,  5.434,  0.000]
cvect =  [ 0.000,  0.000,  5.434]
origin = [ 0.000,  0.000,  0.000]
natoms = 8
natypes = 1
symbols = ('Si',)
pbc = [ True  True  True]
per-atom properties = ['atype', 'pos']
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |   0.000 |   0.000 |   0.000
      1 |       1 |   2.717 |   2.717 |   0.000
      2 |       1 |   2.717 |   0.000 |   2.717
      3 |       1 |   0.000 |   2.717 |   2.717
      4 |       1 |   1.358 |   1.358 |   1.358
      5 |       1 |   1.358 |   4.075 |   4.075
      6 |       1 |   4.075 |   1.358 |   4.075
      7 |       1 |   4.075 |   4.075 |   1.358


#### 1.1.4. Assign forces

Forces are per-atom properties and therefore are to be assigned to the system's atoms.  If the force values are not assigned, the fitting process will simply not use them.

In [5]:
forces = np.array([[ 4.44089210e-16,  3.99680289e-15, -4.44089210e-16],
                   [ 9.19264664e-14,  9.72555370e-14, -1.46410661e-15],
                   [ 9.40914013e-14,  2.22044605e-15,  9.47471268e-14],
                   [-1.58206781e-15,  9.41469125e-14,  9.23705556e-14],
                   [ 3.96904731e-15,  7.10542736e-15,  4.47558657e-15],
                   [-9.92539384e-14, -1.03028697e-13,  7.54951657e-15],
                   [-9.96980276e-14,  3.10862447e-15, -1.00364161e-13],
                   [ 7.54951657e-15, -1.02140518e-13, -9.72555370e-14]])

record.system.atoms.force = uc.set_in_units(forces, 'eV/angstrom')

#### 1.1.5. Convert to JSON and save

The fitting code expects a directory containing JSON files for the reference structures. Building and saving the model representation of the reference structure data will create the JSON files using the correct schema.

- set_system_attributes() extracts symbols, composition and natypes values from the system as separate queryable attributes.
- build_model() builds the record content based on the set values allowing for it to be saved as either json or xml.

In [7]:
# Set system attributes based on system content
record.set_system_attributes()

# Create the JSON record
with open(f'{record.name}.json', 'w') as f:
    record.build_model().json(fp=f)

# Show the generated JSON contents to prove it worked...
with open(f'{record.name}.json') as f:
    print(f.read())

{"reference-structure": {"system-info": {"symbol": "Si", "composition": "Si", "cell": {"natypes": 1}}, "atomic-system": {"box": {"avect": {"value": [5.4335655707252, 0.0, 0.0]}, "bvect": {"value": [0.0, 5.4335601371651, 0.0]}, "cvect": {"value": [0.0, 0.0, 5.4335601371651]}, "origin": {"value": [0.0, 0.0, 0.0]}}, "periodic-boundary-condition": [true, true, true], "atom-type-symbol": "Si", "atoms": {"natoms": 8, "property": [{"name": "pos", "data": {"value": [0.0, 0.0, 0.0, 2.7167827853626, 2.71678006858255, 0.0, 2.7167827853626, 0.0, 2.71678006858255, 0.0, 2.71678006858255, 2.71678006858255, 1.3583913926813, 1.358390034291275, 1.358390034291275, 1.3583913926813, 4.075170102873825, 4.075170102873825, 4.0751741780439, 1.358390034291275, 4.075170102873825, 4.0751741780439, 4.075170102873825, 1.358390034291275], "shape": [8, 3], "unit": "angstrom"}}, {"name": "force", "data": {"value": [4.4408921e-16, 3.99680289e-15, -4.4408921e-16, 9.19264664e-14, 9.7255537e-14, -1.46410661e-15, 9.4091401

#### 1.1.6. Compact definition

Values can alternatively be passed in during record initialization/loading allowing for more concise record creation.

In [8]:
# Load system and define forces
system = am.load('poscar', poscar)
system.atoms.force = uc.set_in_units(forces, 'eV/angstrom')

# Create record and specify all values
record = yabadaba.load_record('reference_structure', name = 'test-define',
                              E_pot_total = uc.set_in_units(-37.039999835953154, 'eV'),
                              E_pot_atom = uc.set_in_units(-4.629999979494144, 'eV'),
                              P_xx = uc.set_in_units(-0.17257378435893145, 'MPa'),
                              P_yy = uc.set_in_units(-0.0645693145365454, 'MPa'),
                              P_zz = uc.set_in_units(-0.06456931454719021, 'MPa'),
                              system = system)

# Set system attributes based on system content
record.set_system_attributes()

# Create the JSON record
with open(f'{record.name}.json', 'w') as f:
    record.build_model().json(fp=f)

# Show the generated JSON contents to prove it worked...
with open(f'{record.name}.json') as f:
    print(f.read())

{"reference-structure": {"system-info": {"symbol": "Si", "composition": "Si", "cell": {"natypes": 1}}, "atomic-system": {"box": {"avect": {"value": [5.4335655707252, 0.0, 0.0]}, "bvect": {"value": [0.0, 5.4335601371651, 0.0]}, "cvect": {"value": [0.0, 0.0, 5.4335601371651]}, "origin": {"value": [0.0, 0.0, 0.0]}}, "periodic-boundary-condition": [true, true, true], "atom-type-symbol": "Si", "atoms": {"natoms": 8, "property": [{"name": "pos", "data": {"value": [0.0, 0.0, 0.0, 2.7167827853626, 2.71678006858255, 0.0, 2.7167827853626, 0.0, 2.71678006858255, 0.0, 2.71678006858255, 2.71678006858255, 1.3583913926813, 1.358390034291275, 1.358390034291275, 1.3583913926813, 4.075170102873825, 4.075170102873825, 4.0751741780439, 1.358390034291275, 4.075170102873825, 4.0751741780439, 4.075170102873825, 1.358390034291275], "shape": [8, 3], "unit": "angstrom"}}, {"name": "force", "data": {"value": [4.4408921e-16, 3.99680289e-15, -4.4408921e-16, 9.19264664e-14, 9.7255537e-14, -1.46410661e-15, 9.4091401

### 1.2. Data transformations

#### 1.2.1. Data transformation methods

The record information can also be transformed into a variety of different formats using class methods and attributes
- __model__ returns the model contents if built.
- __metadata()__ returns a flat dict containing simple metadata fields (useful for parsing and database queries).
- __reference_dict()__ returns a flat dict containing the reference energies, forces and pressures to be used in the fitting code.

In [9]:
# Flat JSON representation
print(record.model.json())

{"reference-structure": {"system-info": {"symbol": "Si", "composition": "Si", "cell": {"natypes": 1}}, "atomic-system": {"box": {"avect": {"value": [5.4335655707252, 0.0, 0.0]}, "bvect": {"value": [0.0, 5.4335601371651, 0.0]}, "cvect": {"value": [0.0, 0.0, 5.4335601371651]}, "origin": {"value": [0.0, 0.0, 0.0]}}, "periodic-boundary-condition": [true, true, true], "atom-type-symbol": "Si", "atoms": {"natoms": 8, "property": [{"name": "pos", "data": {"value": [0.0, 0.0, 0.0, 2.7167827853626, 2.71678006858255, 0.0, 2.7167827853626, 0.0, 2.71678006858255, 0.0, 2.71678006858255, 2.71678006858255, 1.3583913926813, 1.358390034291275, 1.358390034291275, 1.3583913926813, 4.075170102873825, 4.075170102873825, 4.0751741780439, 1.358390034291275, 4.075170102873825, 4.0751741780439, 4.075170102873825, 1.358390034291275], "shape": [8, 3], "unit": "angstrom"}}, {"name": "force", "data": {"value": [4.4408921e-16, 3.99680289e-15, -4.4408921e-16, 9.19264664e-14, 9.7255537e-14, -1.46410661e-15, 9.4091401

In [12]:
# Metadata fields: Note that units are listed in the keys and may differ from the working units!
print(record.metadata())

{'name': 'test-define', 'symbols': ('Si',), 'composition': 'Si', 'natypes': 1, 'natoms': 8, 'E_pot_total (eV)': np.float64(-37.039999835953154), 'E_pot_atom (eV)': np.float64(-4.629999979494144), 'P_xx (GPa)': np.float64(-0.00017257378435893146), 'P_yy (GPa)': np.float64(-6.456931453654541e-05), 'P_zz (GPa)': np.float64(-6.456931454719023e-05)}


In [13]:
# Reference values for use in the fitting code.
print(record.reference_dict())

{'E_pot_total': -37.039999835953154, 'E_pot_atom': -4.629999979494144, 'F': array([[ 4.44089210e-16,  3.99680289e-15, -4.44089210e-16],
       [ 9.19264664e-14,  9.72555370e-14, -1.46410661e-15],
       [ 9.40914013e-14,  2.22044605e-15,  9.47471268e-14],
       [-1.58206781e-15,  9.41469125e-14,  9.23705556e-14],
       [ 3.96904731e-15,  7.10542736e-15,  4.47558657e-15],
       [-9.92539384e-14, -1.03028697e-13,  7.54951657e-15],
       [-9.96980276e-14,  3.10862447e-15, -1.00364161e-13],
       [ 7.54951657e-15, -1.02140518e-13, -9.72555370e-14]]), 'P_xx': -1.0771208410903055e-06, 'P_yy': -4.0300996261155935e-07, 'P_zz': -4.030099626779991e-07}


#### 1.2.2. Supporting methods

The Python class attributes and the model record contents are two independent full representations of the reference structure data.  If you add/update content in one of the representations, you can use the methods below to ensure that the alternate representation is also updated. 

- load_model() reads in model contents from a string or file, updates model and the class attribute values.
- build_model() generates new model contents based on the current class attribute values. The model contents are also returned for convenience.
- reload_model() updates the class attribute values based on the current model contents. This allows for manipulation of the model contents directly.


### 1.3. Database features

ReferenceStructure is a child of yabadaba.record.Record giving it database interaction capabilities. This makes it possible for more advanced handling of the reference structure data...

## 2. Alternate formats

The iprPy_fit code reads in all reference structure data and converts everything into simple formats prior to fitting. As such, it should be easy to interface with alternate data formats as long as converters to the simple representations are created.

The fitting code is looking for these values as two separate dicts: a reference_dict and a system_dict.

### 2.1. Reference values

The reference/target value information is expected as a dict containing at least one of the following keys
- __E_pot_system__ (float) Total system potential energy in eV.
- __E_pot_atom__ (float) Average potential energy per atom in eV.
- __F__ (numpy array) (N,3) array of atomic forces in eV/angstrom.
- __P_xx__ (float) X normal pressure in eV/angstrom^3.
- __P_yy__ (float) Y normal pressure in eV/angstrom^3.
- __P_zz__ (float) Z normal pressure in eV/angstrom^3.

This should match with what is returned by record.reference_dict()

In [14]:
# Reference values for use in the fitting code.
print(record.reference_dict())

{'E_pot_total': -37.039999835953154, 'E_pot_atom': -4.629999979494144, 'F': array([[ 4.44089210e-16,  3.99680289e-15, -4.44089210e-16],
       [ 9.19264664e-14,  9.72555370e-14, -1.46410661e-15],
       [ 9.40914013e-14,  2.22044605e-15,  9.47471268e-14],
       [-1.58206781e-15,  9.41469125e-14,  9.23705556e-14],
       [ 3.96904731e-15,  7.10542736e-15,  4.47558657e-15],
       [-9.92539384e-14, -1.03028697e-13,  7.54951657e-15],
       [-9.96980276e-14,  3.10862447e-15, -1.00364161e-13],
       [ 7.54951657e-15, -1.02140518e-13, -9.72555370e-14]]), 'P_xx': -1.0771208410903055e-06, 'P_yy': -4.0300996261155935e-07, 'P_zz': -4.030099626779991e-07}


### 2.2. System values

The system information should be extracted out as a dict containing the following keys

- __pbc__ (list) A list of 3 strings indicating the LAMMPS boundary conditions, e.g. \['p', 'p', 'p'\] for all periodic boundaries.
- __region_params__ (list) A list of 9 floats that define the box using LAMMPS lo/hi and tilt parameters: \[xlo, xhi, ylo, yhi, zlo, zhi, xy, xz, yz\].
- __atype__ (list or array) The int atom types for each atom.
- __x__ (list or array) The atomic positions. Can be given either as an (N,3) array or as a flattened array.

The iprPy_fit.lammps.dump_lammps_dynamic_parameters() function will extract these and other parameters from an atomman.System.

_Future note:_ The dump method is to be integrated into atomman as the 'lammps_dynamic_parameters' dump style.